In [64]:
import glob 
import xarray as xr
import matplotlib.pyplot as plt


In [62]:
path = "/home/hanna/lagrings/satelite_data/"
save_dir = "/home/hanna/images_simple_statsmodels/"

In [9]:
files= glob.glob(path+"*heilt*.nc")

In [10]:
len(files)

99

In [11]:
for i, fil in enumerate(files):
    if i == 0:
        ds = xr.open_dataset(fil)
    else:
        try:
            ds = ds.merge( xr.open_dataset(fil) )
        except xr.MergeError:
            #ds = ds.merge( xr.open_dataset(fil))
            print(fil)

In [12]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161, time: 921)
Coordinates:
  * time       (time) datetime64[ns] 2004-04-03T08:00:00 ... 2004-08-12T03:00:00
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
Data variables:
    tcc        (time, latitude, longitude) float64 0.1812 0.2106 ... 1.0 1.0
    nr_nans    (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

## Sammenligne med era5

In [13]:
path = "/home/hanna/lagrings/dataERA5_grib/"

In [14]:
f1 = glob.glob(path+"*total*cover*.grib")

In [15]:
f1

['/home/hanna/lagrings/dataERA5_grib/total_cloud_cover2012_2018_surface_level.grib',
 '/home/hanna/lagrings/dataERA5_grib/total_cloud_cover2004_2011_surface_level.grib']

In [16]:
dtcc = xr.open_dataset(f1[-1], engine = "cfgrib")

In [17]:
dtcc

<xarray.Dataset>
Dimensions:     (latitude: 103, longitude: 178, time: 70128)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2004-01-01 ... 2011-12-31T23:00:00
    step        timedelta64[ns] ...
    surface     int64 ...
  * latitude    (latitude) float64 55.5 55.25 55.0 54.75 ... 30.5 30.25 30.0
  * longitude   (longitude) float64 -15.0 -14.75 -14.5 ... 28.75 29.0 29.25
    valid_time  (time) datetime64[ns] ...
Data variables:
    tcc         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2019-11-24T08:25:10 GRIB to CDM+CF via cfgrib-0....

In [9]:
f1

'/home/hanna/lagrings/dataERA5_grib/total_cloud_cover2012_2018_surface_level.grib'

In [18]:
de = dtcc.sel(time = ds.time)

In [40]:
de.isel(time = 0).tcc.values

array([[0.9255676 , 0.929718  , 0.92974854, ..., 0.        , 0.        ,
        0.        ],
       [0.9656067 , 0.9356079 , 0.91156006, ..., 0.        , 0.        ,
        0.        ],
       [0.9700012 , 0.9273987 , 0.89349365, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.95184326, 0.95962524, 0.9605713 , ..., 0.5704346 , 0.67904663,
        0.6309204 ],
       [0.95877075, 0.8789673 , 0.8661499 , ..., 0.6826782 , 0.6854553 ,
        0.75994873],
       [0.9300232 , 0.8339844 , 0.7799988 , ..., 0.48251343, 0.523468  ,
        0.5554199 ]], dtype=float32)

In [41]:
de2 = de.sel(latitude = slice(50,30) , longitude = slice(-15, 25))

In [43]:
#de2.tcc.values

In [31]:
#ds['tcc']

In [44]:
#de2['tcc'].isel(time = 0)

In [45]:
#de2['tcc'].plot()''

<xarray.Dataset>
Dimensions:     (latitude: 81, longitude: 161, time: 921)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2004-04-03T08:00:00 ... 2004-08-12T03:00:00
    step        timedelta64[ns] ...
    surface     int64 ...
  * latitude    (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
  * longitude   (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    era         (time, latitude, longitude) float32 0.60165405 ... 0.9625244
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2019-11-24T08:25:10 GRIB to CDM+CF via cfgrib-0....

In [53]:
merged = xr.merge([de2.rename({'tcc':'era'}), ds['tcc']])

In [63]:
merged.to_netcdf(save_dir + 'diff_era_calc_frac.nc')

In [55]:
diff = merged['era']-merged['tcc']

In [56]:
diff

<xarray.DataArray (time: 921, latitude: 81, longitude: 161)>
array([[[ 0.74882252,  0.62337493,  0.77341353, ..., -0.86315863,
         -0.71737435, -0.79760821],
        [ 0.30702848,  0.11677554,  0.27912357, ..., -0.77530652,
         -0.91781616, -0.51948089],
        [ 0.60408447,  0.88719974,  0.50864167, ..., -0.74154663,
         -0.68215942, -0.14219849],
        ...,
        [ 0.98086548,  0.98562622,  0.41821072, ..., -0.72092798,
         -0.6254758 , -0.81248681],
        [ 0.85931396,  0.91357422,  0.64008474, ..., -0.79045899,
         -0.72248033, -0.61115065],
        [ 0.60165405,  0.662323  ,  0.75457764, ..., -0.51680384,
         -0.72523014, -0.59245532]],

       [[ 0.91547833,  0.45018165, -0.20976703, ..., -0.03636397,
         -0.036115  , -0.85228335],
        [ 0.74094789,  0.39786788, -0.10689465, ..., -0.1007522 ,
          0.14717203, -0.32799586],
        [ 0.36206363,  0.62010673,  0.85550088, ..., -0.49641443,
         -0.07506933,  0.10923201],
      